In [1]:
import pandas as pd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
load_dotenv()
import os
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.alp_client_extractor import ALPClientExtractor
from database.adatabase import ADatabase
from multiprocessing import Pool

In [2]:
crypto_list = ["AAVE", "AVAX", "BAT", "BCH", 
 "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI", "UNI", "XTZ", "YFI"]
alp = ALPClientExtractor(os.getenv("APCAKEY"),os.getenv("APCASECRET"))

In [3]:
market = ADatabase("market")
market.connect()
index = market.retrieve("sp500")
market.disconnect()

In [15]:
start = datetime(2024,1,1)
end = datetime.now()

In [16]:
prices = []
date = start
while date < end:
    for ticker in crypto_list:
        try:
            price = alp.crypto(ticker+"/USD",date,date+timedelta(minutes=10000))
            price.sort_values("date",inplace=True)
            price["date"] = pd.to_datetime(price["date"])
            date = date + timedelta(minutes=10000)
            prices.append(price)
        except Exception as e:
            print(str(e))

'bars'


In [17]:
market_prices = pd.concat(prices)

In [35]:
def calculate_indicators(price, timeframe):
    """Calculate indicators for a single ticker."""
    price = price.sort_values("date")
    price["sma"] = price["adjclose"].rolling(timeframe).mean()
    price["ema"] = price["adjclose"].ewm(span=timeframe, adjust=False).mean()

    delta = price["adjclose"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(timeframe).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(timeframe).mean()
    rs = gain / loss
    price["rsi"] = 100 - (100 / (1 + rs))

    price["std"] = price["adjclose"].rolling(timeframe).std()
    price["bollinger_upper"] = price["sma"] + 2 * price["std"]
    price["bollinger_lower"] = price["sma"] - 2 * price["std"]

    price["pct_change"] = price["adjclose"].pct_change(periods=timeframe)
    price["momentum"] = price["adjclose"] - price["adjclose"].shift(timeframe)

    price["volatility"] = price["adjclose"].rolling(timeframe).std()
    price["coev"] = price["adjclose"].rolling(timeframe).std() / price["adjclose"].rolling(timeframe).mean()

    return price.dropna()

In [65]:
signals = ["sma"
           ,"ema","rsi","volatility","bollinger_upper","bollinger_lower","momentum","pct_change","coev"
          ]
ascendings = [True,False]
timeframes = [960, 1920]
base = {"cash":100,"ticker":"","adjclose":0,"quantity":0,"buy_price":0}

In [87]:
def simulate_portfolio(sim, dates, signal, ascending, timeframe, base):
    """Simulate portfolio for a given signal and timeframe."""
    portfolios = []
    portfolio = base.copy()
    for date in dates[::timeframe]:
        try:
            today = sim[sim["date"] == date]
            max_return = max(0.005, today["pct_change"].mean())
            min_return = today["pct_change"].mean()
            portfolio["date"] = date
            if portfolio["ticker"] != "":
                ticker_data = today[today["ticker"] == portfolio["ticker"]]
                if not ticker_data.empty:
                    portfolio["adjclose"] = ticker_data["adjclose"].iloc[0]
                    pnl = portfolio["adjclose"] / portfolio["buy_price"] - 1
                    if pnl >= max_return or pnl <= min_return:
                        cash = portfolio["adjclose"] * portfolio["quantity"] * 0.9985
                        portfolio = base.copy()
                        portfolio["date"] = date
                        portfolio["cash"] = cash
            else:
                if portfolio["cash"] != 0:
                    opportunity = today.sort_values(signal, ascending=ascending).iloc[0]
                    portfolio.update({
                        "ticker": opportunity["ticker"],
                        "adjclose": opportunity["adjclose"],
                        "buy_price": opportunity["adjclose"],
                        "quantity": portfolio["cash"] / opportunity["adjclose"] * 0.9975,
                        "cash": 0,
                    })
            portfolios.append(portfolio.copy())
        except Exception as e:
            print(str(e))
    result = pd.DataFrame(portfolios)
    result["pv"] = result["cash"] + result["adjclose"] * result["quantity"]
    return {
        "signal": signal,
        "ascending": ascending,
        "timeframe": timeframe,
        "pv": result["pv"].iloc[-1],
        "risk":result["pv"].std()
    }

def process_signal(args):
    sim, dates, signal, ascending, timeframe, base = args
    return simulate_portfolio(sim, dates, signal, ascending, timeframe, base)

def process_timeframe(timeframe, crypto_list, market_prices, signals, ascendings, base):
    """Prepare simulation data and process signals sequentially."""
    # Calculate indicators for all tickers in the timeframe
    prices = []
    for ticker in crypto_list:
        try:
            price = market_prices[market_prices["ticker"] == ticker + "/USD"]
            price = calculate_indicators(price, timeframe)
            prices.append(price)
        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    # Combine and sort all price data
    sim = pd.concat(prices).dropna().sort_values("date")
    dates = sim["date"].unique()

    # Process each signal sequentially
    results = []
    for signal in signals:
        for ascending in ascendings:
            args = (sim, dates, signal, ascending, timeframe, base)
            try:
                result = process_signal(args)
                results.append(result)
            except Exception as e:
                print(f"Error processing signal {signal}, ascending={ascending}: {e}")

    return results

In [88]:
all_results = []
for timeframe in tqdm(timeframes):
    all_results += process_timeframe(timeframe, crypto_list, market_prices, signals, ascendings, base)
analysis = pd.DataFrame(all_results)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.21s/it]


In [89]:
analysis.sort_values("pv",ascending=False).head(50)

,signal,ascending,timeframe,pv,risk
18,sma,True,15,88.834428,3.008230
27,bollinger_upper,False,15,88.834428,3.008230
21,ema,False,15,88.834428,3.008230
22,rsi,True,15,88.834428,3.008230
23,rsi,False,15,88.834428,3.008230
24,volatility,True,15,88.834428,3.008230
25,volatility,False,15,88.834428,3.008230
26,bollinger_upper,True,15,88.834428,3.008230
28,bollinger_lower,True,15,88.834428,3.008230
19,sma,False,15,88.834428,3.008230
